In [1]:
from custom_dataset import Dataset_Binary, custom_collate
from models import *

# LOAD DATA
data_dir = '../labeled_data/'
labeled_data = os.listdir(data_dir)
for ni,i in sorted(zip(labeled_data,range(len(labeled_data)))):print(i,'\t',ni)

data = labeled_data[41]
print('\n',data)
data_df = pd.read_csv(data_dir+data)

nlabels = len(data_df['int_label'].unique())
device = 'cuda:1'

# Create dataset
dataset = Dataset_Binary(data_df, device)

# Create DataLoader
# dataloader = DataLoader(dataset, batch_size=100, shuffle=False, collate_fn=custom_collate)

36 	 BINARYdata_CANCER_TYPE_0MinMutations_1696MinCancerType.csv
24 	 BINARYdata_CANCER_TYPE_0MinMutations_169MinCancerType.csv
2 	 BINARYdata_CANCER_TYPE_3MinMutations_1696MinCancerType.csv
32 	 BINARYdata_CANCER_TYPE_3MinMutations_169MinCancerType.csv
13 	 BINARYdata_CANCER_TYPE_DETAILED_0MinMutations_1696MinCancerType.csv
26 	 BINARYdata_CANCER_TYPE_DETAILED_0MinMutations_169MinCancerType.csv
11 	 BINARYdata_CANCER_TYPE_DETAILED_3MinMutations_1696MinCancerType.csv
30 	 BINARYdata_CANCER_TYPE_DETAILED_3MinMutations_169MinCancerType.csv
40 	 canonical_ref_barcode_refIdxs_map.pkl
23 	 data_CANCER_TYPE_0MinMutations_1696MinCancerType.csv
25 	 data_CANCER_TYPE_0MinMutations_169MinCancerType.csv
44 	 data_CANCER_TYPE_3MinMutations_1696MinCancerType.csv
18 	 data_CANCER_TYPE_3MinMutations_169MinCancerType.csv
12 	 data_CANCER_TYPE_DETAILED_0MinMutations_1696MinCancerType.csv
28 	 data_CANCER_TYPE_DETAILED_0MinMutations_169MinCancerType.csv
3 	 data_CANCER_TYPE_DETAILED_3MinMutations_1696Min

In [2]:
dataset[0]

(tensor([0., 0., 0.,  ..., 0., 0., 0.], device='cuda:1'),
 tensor(34, device='cuda:1'))

In [4]:

train_loader, test_loader = getTrainTestLoaders(dataset)
class Config:
    input_dim: int = 1448
    bias: bool = False
    n_labels: int = 17

print('n labels:',nlabels)
config = Config()
config.n_labels = nlabels

model = MLPClassifier(config)
model.to(device)

num_epochs = 3
learning_rate = 0.001

train(model,num_epochs,train_loader,test_loader)

n labels: 53


TESTING: 100%|██████████| 230/230 [00:00<00:00, 319.50it/s]


Test Accuracy: 48.54%, (11159 of 22990)


TESTING: 100%|██████████| 230/230 [00:00<00:00, 324.29it/s]


Test Accuracy: 50.03%, (11501 of 22990)


TESTING: 100%|██████████| 230/230 [00:00<00:00, 322.93it/s]

Test Accuracy: 49.83%, (11456 of 22990)


In [5]:
import torch.optim as optim
from tqdm import tqdm

class Config:
    input_dim: int = 1448
    n_labels: int = 17

print('n labels:',nlabels)
config = Config()
config.n_labels = nlabels

model = LRClassifier(config)
model.to(device)

num_epochs = 3
learning_rate = 0.001

train(model,num_epochs,train_loader,test_loader)

n labels: 53


TESTING: 100%|██████████| 230/230 [00:00<00:00, 322.15it/s]


Test Accuracy: 41.02%, (9430 of 22990)


TESTING: 100%|██████████| 230/230 [00:00<00:00, 329.68it/s]


Test Accuracy: 46.49%, (10687 of 22990)


TESTING: 100%|██████████| 230/230 [00:00<00:00, 328.98it/s]

Test Accuracy: 47.84%, (10998 of 22990)


In [7]:
# #bug report
# #len(genes) should be 100
# #len(unique_idxs) for data_df_bin is 100
# #likely, there are None mut embeddings mapping to the same gene in ref_map
# flatten = lambda x: [i for j in x for i in j]
# load_str = lambda x: list(map(int,x.split(',')))
# data = [load_str(i) for i in data_df_emb['idxs'].values]
# ref_map = {i[5]:i[4] for j in tumors.values() for i in j for k in i}
# unique_idxs = list(set(flatten(data)))
# genes = list(set([ref_map[i] for i in unique_idxs]))
# len(genes)

In [2]:
#4,9
from custom_dataset import Dataset_TopN_emb
# mut_embeddings = np.load('../aa/canonical_mut_embeddings_esm2.npy')
mut_embeddings = np.load('../aa/canonical_mut_average_embeddings_esm2.npy')
ref_embeddings = np.load('../aa/canonical_ref_embeddings_esm2.npy')
tumors = pickleLoad('../aa/tumors.pkl')
data_bin = labeled_data[0]
data_emb = labeled_data[41]
print(data_bin)
print(data_emb)
data_df_bin = pd.read_csv(data_dir+data_bin)
data_df_emb = pd.read_csv(data_dir+data_emb)

loading data from ../aa/tumors.pkl
top100_BINARYdata_CANCER_TYPE_3MinMutations_160MinCancerType.csv
top100_data_CANCER_TYPE_3MinMutations_169MinCancerType.csv


In [3]:
dataset_bin = Dataset_Binary(data_df_bin, device)
train_loader_bin, test_loader_bin = getTrainTestLoaders(dataset_bin, batch_size = 1000)
dataset_emb = Dataset_TopN_emb(data_df_emb, mut_embeddings, ref_embeddings, tumors, device, pca=64)
train_loader_emb, test_loader_emb = getTrainTestLoaders(dataset_emb, batch_size = 1000)
dataset_att = Dataset_TopN_emb(data_df_emb, mut_embeddings, ref_embeddings, tumors, device, flat = False)
train_loader_att, test_loader_att = getTrainTestLoaders(dataset_att, batch_size = 1000, collate=custom_collate)
nlabels_bin = len(data_df_bin['int_label'].unique())
nlabels_emb = len(data_df_emb['int_label'].unique())
print(nlabels_bin,nlabels_emb)


## HYPERPARAMS
num_epochs = 10
learning_rate = 0.001


209182 unique mutations
98 genes
209182 unique mutations
98 genes
51 53


In [10]:
## ATTENTION MODEL WITH POSITION EMBEDDING
class Config_Att:
    n_layer: int = 3
    input_dim: int = 640
    dropout: float = 0.0
    bias: bool = False
    n_labels: int = 17
    pooling : str = 'mean'
    norm_fn: nn.Module = nn.LayerNorm
    max_len : int = 98
    position_embedding: bool = True

config_att = Config_Att()
config_att.n_labels = nlabels_emb

model_att = Classifier(config_att)
model_att.to(device)

num_epochs = 15
train(model_att,num_epochs,train_loader_att,test_loader_att)

TESTING: 100%|██████████| 18/18 [00:08<00:00,  2.17it/s]


Test Accuracy: 38.75%, (6822 of 17603)


TESTING: 100%|██████████| 18/18 [00:08<00:00,  2.13it/s]


Test Accuracy: 46.30%, (8150 of 17603)


TESTING: 100%|██████████| 18/18 [00:08<00:00,  2.15it/s]


...

Test Accuracy: 54.34%, (9566 of 17603)


TESTING: 100%|██████████| 18/18 [00:08<00:00,  2.12it/s]


Test Accuracy: 54.00%, (9505 of 17603)


TESTING: 100%|██████████| 18/18 [00:08<00:00,  2.09it/s]

Test Accuracy: 54.01%, (9508 of 17603)


In [6]:
## ATTENTION MODEL WITHOUT POSITION EMBEDDING
class Config_Att:
    n_layer: int = 3
    input_dim: int = 640
    dropout: float = 0.0
    bias: bool = False
    n_labels: int = 17
    pooling : str = 'mean'
    norm_fn: nn.Module = nn.LayerNorm
    position_embedding: bool = False
    max_len : int = 100

config_att = Config_Att()
config_att.n_labels = nlabels_emb

model_att = Classifier(config_att)
model_att.to(device)

train(model_att,num_epochs,train_loader_att,test_loader_att)

TESTING: 100%|██████████| 18/18 [00:07<00:00,  2.33it/s]


Test Accuracy: 31.05%, (5465 of 17603)


TESTING: 100%|██████████| 18/18 [00:07<00:00,  2.41it/s]


Test Accuracy: 34.76%, (6118 of 17603)


TESTING: 100%|██████████| 18/18 [00:07<00:00,  2.43it/s]


...

Test Accuracy: 51.64%, (9091 of 17603)


TESTING: 100%|██████████| 18/18 [00:08<00:00,  2.24it/s]


Test Accuracy: 52.48%, (9238 of 17603)


TESTING: 100%|██████████| 18/18 [00:07<00:00,  2.41it/s]

Test Accuracy: 51.82%, (9121 of 17603)


In [5]:
##EMBEDDING MODEL
class Config_Emb:
    input_dim: int = 6272 #98 instead of 100 for some reason
    bias: bool = False
    n_labels: int

print('n labels:',nlabels)
config_emb = Config_Emb()
config_emb.n_labels = nlabels_emb

model_emb = BigMLP(config_emb)
model_emb.to(device)

train(model_emb,num_epochs,train_loader_emb,test_loader_emb)

n labels: 53


TESTING: 100%|██████████| 177/177 [00:01<00:00, 101.91it/s]


Test Accuracy: 52.28%, (9202 of 17603)


TESTING: 100%|██████████| 177/177 [00:01<00:00, 104.57it/s]


Test Accuracy: 53.14%, (9355 of 17603)


...

Test Accuracy: 50.97%, (8973 of 17603)


TESTING: 100%|██████████| 177/177 [00:01<00:00, 105.16it/s]


Test Accuracy: 50.36%, (8864 of 17603)


TESTING: 100%|██████████| 177/177 [00:01<00:00, 107.97it/s]

Test Accuracy: 50.08%, (8815 of 17603)


In [4]:
## BINARY MODEL
class Config_Bin:
    input_dim: int = 1448
    bias: bool = False
    n_labels: int
config_bin = Config_Bin()
config_bin.n_labels = nlabels_bin

model_bin = MLPClassifier(config_bin)
model_bin.to(device)

train(model_bin,num_epochs,train_loader_bin,test_loader_bin)

TESTING: 100%|██████████| 19/19 [00:00<00:00, 36.46it/s]


Test Accuracy: 31.78%, (5786 of 18209)


TESTING: 100%|██████████| 19/19 [00:00<00:00, 37.97it/s]


Test Accuracy: 45.18%, (8227 of 18209)


TESTING: 100%|██████████| 19/19 [00:00<00:00, 37.75it/s]


...

Test Accuracy: 48.78%, (8883 of 18209)


TESTING: 100%|██████████| 19/19 [00:00<00:00, 38.00it/s]


Test Accuracy: 48.78%, (8882 of 18209)


TESTING: 100%|██████████| 19/19 [00:00<00:00, 37.95it/s]

Test Accuracy: 49.38%, (8992 of 18209)
